In [85]:
import numpy as np
import pandas as pd
import os
import dask.dataframe as dd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import ResNet50, mobilenet_v3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

In [86]:
train_folder = "machine-learning-in-science-ii-2024/training_data/training_data/"
test_folder = "machine-learning-in-science-ii-2024/test_data/test_data"
train_df = pd.read_csv('machine-learning-in-science-ii-2024/training_norm.csv')

In [87]:
# Define data loading function
image_size = 224,224
def load_image(image_id):
    image_path = os.path.join(train_folder, f"{image_id}.png")
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image)
    image /= 255.0  # Normalizing to [0, 1]
    return image

In [88]:
load_image('2')

array([[[0.5568628 , 0.5647059 , 0.627451  ],
        [0.5019608 , 0.53333336, 0.6117647 ],
        [0.53333336, 0.5647059 , 0.64705884],
        ...,
        [0.13333334, 0.22352941, 0.2627451 ],
        [0.19215687, 0.28235295, 0.32156864],
        [0.14117648, 0.27450982, 0.32941177]],

       [[0.53333336, 0.57254905, 0.61960787],
        [0.5058824 , 0.5254902 , 0.60784316],
        [0.5529412 , 0.5764706 , 0.65882355],
        ...,
        [0.28235295, 0.2509804 , 0.28235295],
        [0.27450982, 0.24313726, 0.27058825],
        [0.20784314, 0.26666668, 0.2901961 ]],

       [[0.53333336, 0.57254905, 0.6039216 ],
        [0.53333336, 0.5411765 , 0.6117647 ],
        [0.4627451 , 0.47058824, 0.5372549 ],
        ...,
        [0.28235295, 0.25490198, 0.27450982],
        [0.28235295, 0.25490198, 0.27450982],
        [0.25882354, 0.2509804 , 0.3137255 ]],

       ...,

       [[0.77254903, 0.7607843 , 0.7764706 ],
        [0.7529412 , 0.75686276, 0.77254903],
        [0.7607843 , 0

In [89]:
# Convert pandas DataFrame to Dask DataFrame
ddf = dd.from_pandas(train_df, npartitions=4)

In [90]:
# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((ddf['image_id'].values.compute(), 
                                              ddf[['angle', 'speed']].values.compute()))

In [91]:
# Map loading function and preprocess images
def load_and_preprocess_image(image_id, label):
    image = tf.py_function(load_image, [image_id], tf.float32)
    return image, label
dataset = dataset.map(load_and_preprocess_image)

In [92]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(train_df))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [93]:
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=train_size).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [94]:
mobile =  tf.keras.applications.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [95]:
x = mobile.layers[-6].output
output = Dense(units=2, activation = 'softmax')(x)

In [96]:
from tensorflow.keras.models import Model
model = Model(inputs = mobile.input, outputs = output)

In [97]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [98]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow.keras.backend as K

# Load MobileNet base model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Add custom layers on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
angle_norm_output = Dense(1, name='angle_norm')(x)
speed_output = Dense(1, name='speed')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=[angle_norm_output, speed_output])

# Custom mean absolute error metric for angle_norm
def angle_norm_mae(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred))

# Custom mean absolute error metric for speed
def speed_mae(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred))

# Compile the model
model.compile(optimizer='adam',
              loss=['mse', 'mse'],
              metrics={'angle_norm': angle_norm_mae, 'speed': speed_mae})
# Train the model
model.fit(train_dataset,
              epochs=20,
              validation_data=val_dataset)

Epoch 1/20
345/345 [==============================] - 7586s 19s/step - loss: 0.2118 - angle_norm_loss: 0.1094 - speed_loss: 0.1024 - angle_norm_angle_norm_mae: 0.2527 - speed_speed_mae: 0.2523 - val_loss: 0.1488 - val_angle_norm_loss: 0.0691 - val_speed_loss: 0.0797 - val_angle_norm_angle_norm_mae: 0.2279 - val_speed_speed_mae: 0.2319
Epoch 2/20
345/345 [==============================] - 2590s 7s/step - loss: 0.1260 - angle_norm_loss: 0.0628 - speed_loss: 0.0632 - angle_norm_angle_norm_mae: 0.2250 - speed_speed_mae: 0.2253 - val_loss: 0.1323 - val_angle_norm_loss: 0.0634 - val_speed_loss: 0.0689 - val_angle_norm_angle_norm_mae: 0.2251 - val_speed_speed_mae: 0.2258
Epoch 3/20
345/345 [==============================] - 3445s 9s/step - loss: 0.1216 - angle_norm_loss: 0.0607 - speed_loss: 0.0609 - angle_norm_angle_norm_mae: 0.2235 - speed_speed_mae: 0.2236 - val_loss: 0.1272 - val_angle_norm_loss: 0.0633 - val_speed_loss: 0.0638 - val_angle_norm_angle_norm_mae: 0.2243 - val_speed_speed_mae

In [81]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['mse'])

In [ ]:
import matplotlib.pyplot as plt

# Make predictions
predictions = model.predict(X_val)
predicted_angles = predictions[0].flatten()  # Adjust indexing based on your model's output
predicted_speeds = predictions[1].flatten()

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

# Plot for angle
axs[0].scatter(y_angle_val, predicted_angles, alpha=0.3)
axs[0].plot([y_angle_val.min(), y_angle_val.max()], [y_angle_val.min(), y_angle_val.max()], 'k--', lw=4)
axs[0].set_xlabel('Measured')
axs[0].set_ylabel('Predicted')
axs[0].set_title('Angle Prediction')

# Plot for speed
axs[1].scatter(y_speed_val, predicted_speeds, alpha=0.3)
axs[1].plot([y_speed_val.min(), y_speed_val.max()], [y_speed_val.min(), y_speed_val.max()], 'k--', lw=4)
axs[1].set_xlabel('Measured')
axs[1].set_ylabel('Predicted')
axs[1].set_title('Speed Prediction')

plt.tight_layout()
plt.show()

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (224, 224, 3)  # Adjust based on your preprocessing

inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
# Add more layers as needed...
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='sigmoid')(x)
x = Dropout(0.5)(x)

# Output layers
angle_output = Dense(1, name='angle_output')(x)
speed_output = Dense(1, name='speed_output')(x)

model = Model(inputs=inputs, outputs=[angle_output, speed_output])

model.compile(optimizer='adam', loss={'angle_output': 'mse', 'speed_output': 'mse'}, metrics={'angle_output': 'mae', 'speed_output': 'mae'})

In [10]:
history2 = model.fit(X_train, {'angle_output': y_angle_train, 'speed_output': y_speed_train}, validation_data=(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val}), epochs=10, batch_size=32)

Epoch 1/10
345/345 [==============================] - 603s 2s/step - loss: 1.2807 - angle_output_loss: 0.6603 - speed_output_loss: 0.6205 - angle_output_mae: 0.6163 - speed_output_mae: 0.6238 - val_loss: 0.2821 - val_angle_output_loss: 0.0806 - val_speed_output_loss: 0.2016 - val_angle_output_mae: 0.2379 - val_speed_output_mae: 0.3364
Epoch 2/10
345/345 [==============================] - 450s 1s/step - loss: 0.4286 - angle_output_loss: 0.1421 - speed_output_loss: 0.2865 - angle_output_mae: 0.2925 - speed_output_mae: 0.4486 - val_loss: 0.2142 - val_angle_output_loss: 0.0254 - val_speed_output_loss: 0.1887 - val_angle_output_mae: 0.1255 - val_speed_output_mae: 0.3719
Epoch 3/10
345/345 [==============================] - 495s 1s/step - loss: 0.2931 - angle_output_loss: 0.0652 - speed_output_loss: 0.2279 - angle_output_mae: 0.1975 - speed_output_mae: 0.4093 - val_loss: 0.2188 - val_angle_output_loss: 0.0260 - val_speed_output_loss: 0.1928 - val_angle_output_mae: 0.1317 - val_speed_output_m

In [11]:
# Evaluate the model on the validation set
eval_results2 = model.evaluate(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val})

# Assuming the model's metrics were defined in the same order as the outputs: ['mse', 'mae'] for each output
angle_mse = eval_results2[1]  # This index might change based on how Keras returns eval results
speed_mse = eval_results2[2]  # Adjust index based on actual results

print(f"Angle MSE: {angle_mse}")
print(f"Speed MSE: {speed_mse}")

87/87 [==============================] - 28s 320ms/step - loss: 0.2135 - angle_output_loss: 0.0248 - speed_output_loss: 0.1887 - angle_output_mae: 0.1274 - speed_output_mae: 0.3720
Angle MSE: 0.02478850819170475
Speed MSE: 0.1887403279542923


In [12]:
history3 = model.fit(X_train, {'angle_output': y_angle_train, 'speed_output': y_speed_train}, validation_data=(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val}), epochs=50, batch_size=64)

Epoch 1/50
173/173 [==============================] - 457s 3s/step - loss: 0.2100 - angle_output_loss: 0.0246 - speed_output_loss: 0.1854 - angle_output_mae: 0.1265 - speed_output_mae: 0.3714 - val_loss: 0.2134 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.1886 - val_angle_output_mae: 0.1277 - val_speed_output_mae: 0.3785
Epoch 2/50
173/173 [==============================] - 458s 3s/step - loss: 0.2094 - angle_output_loss: 0.0245 - speed_output_loss: 0.1849 - angle_output_mae: 0.1260 - speed_output_mae: 0.3701 - val_loss: 0.2135 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.1887 - val_angle_output_mae: 0.1269 - val_speed_output_mae: 0.3724
Epoch 3/50
173/173 [==============================] - 446s 3s/step - loss: 0.2096 - angle_output_loss: 0.0243 - speed_output_loss: 0.1852 - angle_output_mae: 0.1255 - speed_output_mae: 0.3708 - val_loss: 0.2134 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.1886 - val_angle_output_mae: 0.1276 - val_speed_output_m

173/173 [==============================] - 345s 2s/step - loss: 0.2092 - angle_output_loss: 0.0243 - speed_output_loss: 0.1849 - angle_output_mae: 0.1256 - speed_output_mae: 0.3674 - val_loss: 0.2135 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.1887 - val_angle_output_mae: 0.1269 - val_speed_output_mae: 0.3736
Epoch 50/50
173/173 [==============================] - 340s 2s/step - loss: 0.2094 - angle_output_loss: 0.0243 - speed_output_loss: 0.1851 - angle_output_mae: 0.1256 - speed_output_mae: 0.3695 - val_loss: 0.2135 - val_angle_output_loss: 0.0249 - val_speed_output_loss: 0.1887 - val_angle_output_mae: 0.1282 - val_speed_output_mae: 0.3800


In [13]:
# Evaluate the model on the validation set
eval_results3 = model.evaluate(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val})

# Assuming the model's metrics were defined in the same order as the outputs: ['mse', 'mae'] for each output
angle_mse = eval_results3[1]  # This index might change based on how Keras returns eval results
speed_mse = eval_results3[2]  # Adjust index based on actual results

print(f"Angle MSE: {angle_mse}")
print(f"Speed MSE: {speed_mse}")

87/87 [==============================] - 21s 238ms/step - loss: 0.2135 - angle_output_loss: 0.0249 - speed_output_loss: 0.1887 - angle_output_mae: 0.1282 - speed_output_mae: 0.3800
Angle MSE: 0.02486049197614193
Speed MSE: 0.18865789473056793


In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (224, 224, 3)  # Adjust based on your preprocessing

inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
# Add more layers as needed...
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='sigmoid')(x)
x = Dropout(0.5)(x)

# Output layers
angle_output = Dense(1, name='angle_output')(x)
speed_output = Dense(1, name='speed_output')(x)

model = Model(inputs=inputs, outputs=[angle_output, speed_output])

model.compile(optimizer='SGD', loss={'angle_output': 'mse', 'speed_output': 'mse'}, metrics={'angle_output': 'mae', 'speed_output': 'mae'})

In [15]:
history4 = model.fit(X_train, {'angle_output': y_angle_train, 'speed_output': y_speed_train}, validation_data=(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val}), epochs=50, batch_size=64)

Epoch 1/50
173/173 [==============================] - 255s 1s/step - loss: 0.5652 - angle_output_loss: 0.2035 - speed_output_loss: 0.3617 - angle_output_mae: 0.3452 - speed_output_mae: 0.4786 - val_loss: 0.2154 - val_angle_output_loss: 0.0262 - val_speed_output_loss: 0.1892 - val_angle_output_mae: 0.1316 - val_speed_output_mae: 0.3676
Epoch 2/50
173/173 [==============================] - 245s 1s/step - loss: 0.2903 - angle_output_loss: 0.0620 - speed_output_loss: 0.2283 - angle_output_mae: 0.1975 - speed_output_mae: 0.3978 - val_loss: 0.2202 - val_angle_output_loss: 0.0279 - val_speed_output_loss: 0.1922 - val_angle_output_mae: 0.1375 - val_speed_output_mae: 0.4071
Epoch 3/50
173/173 [==============================] - 268s 2s/step - loss: 0.2501 - angle_output_loss: 0.0429 - speed_output_loss: 0.2072 - angle_output_mae: 0.1650 - speed_output_mae: 0.3853 - val_loss: 0.2154 - val_angle_output_loss: 0.0254 - val_speed_output_loss: 0.1901 - val_angle_output_mae: 0.1300 - val_speed_output_m

173/173 [==============================] - 268s 2s/step - loss: 0.1375 - angle_output_loss: 0.0241 - speed_output_loss: 0.1134 - angle_output_mae: 0.1253 - speed_output_mae: 0.2599 - val_loss: 0.1190 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0942 - val_angle_output_mae: 0.1276 - val_speed_output_mae: 0.2425
Epoch 50/50
173/173 [==============================] - 229s 1s/step - loss: 0.1339 - angle_output_loss: 0.0241 - speed_output_loss: 0.1098 - angle_output_mae: 0.1253 - speed_output_mae: 0.2559 - val_loss: 0.1167 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0919 - val_angle_output_mae: 0.1279 - val_speed_output_mae: 0.2331


In [16]:
# Evaluate the model on the validation set
eval_results4 = model.evaluate(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val})

# Assuming the model's metrics were defined in the same order as the outputs: ['mse', 'mae'] for each output
angle_mse = eval_results3[1]  # This index might change based on how Keras returns eval results
speed_mse = eval_results3[2]  # Adjust index based on actual results

print(f"Angle MSE: {angle_mse}")
print(f"Speed MSE: {speed_mse}")

87/87 [==============================] - 26s 300ms/step - loss: 0.1167 - angle_output_loss: 0.0248 - speed_output_loss: 0.0919 - angle_output_mae: 0.1279 - speed_output_mae: 0.2331
Angle MSE: 0.02486049197614193
Speed MSE: 0.18865789473056793


In [17]:
history5 = model.fit(X_train, {'angle_output': y_angle_train, 'speed_output': y_speed_train}, validation_data=(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val}), epochs=100, batch_size=64)

Epoch 1/100
173/173 [==============================] - 356s 2s/step - loss: 0.1317 - angle_output_loss: 0.0241 - speed_output_loss: 0.1075 - angle_output_mae: 0.1254 - speed_output_mae: 0.2522 - val_loss: 0.1862 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.1614 - val_angle_output_mae: 0.1270 - val_speed_output_mae: 0.3705
Epoch 2/100
173/173 [==============================] - 377s 2s/step - loss: 0.1296 - angle_output_loss: 0.0241 - speed_output_loss: 0.1055 - angle_output_mae: 0.1254 - speed_output_mae: 0.2509 - val_loss: 0.1048 - val_angle_output_loss: 0.0247 - val_speed_output_loss: 0.0800 - val_angle_output_mae: 0.1269 - val_speed_output_mae: 0.2132
Epoch 3/100
173/173 [==============================] - 335s 2s/step - loss: 0.1280 - angle_output_loss: 0.0241 - speed_output_loss: 0.1039 - angle_output_mae: 0.1254 - speed_output_mae: 0.2474 - val_loss: 0.1052 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0804 - val_angle_output_mae: 0.1273 - val_speed_outpu

173/173 [==============================] - 221s 1s/step - loss: 0.0797 - angle_output_loss: 0.0241 - speed_output_loss: 0.0556 - angle_output_mae: 0.1254 - speed_output_mae: 0.1768 - val_loss: 0.0734 - val_angle_output_loss: 0.0249 - val_speed_output_loss: 0.0485 - val_angle_output_mae: 0.1252 - val_speed_output_mae: 0.1348
Epoch 50/100
173/173 [==============================] - 214s 1s/step - loss: 0.0763 - angle_output_loss: 0.0241 - speed_output_loss: 0.0522 - angle_output_mae: 0.1253 - speed_output_mae: 0.1721 - val_loss: 0.0726 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0478 - val_angle_output_mae: 0.1261 - val_speed_output_mae: 0.1323
Epoch 51/100
173/173 [==============================] - 243s 1s/step - loss: 0.0765 - angle_output_loss: 0.0241 - speed_output_loss: 0.0524 - angle_output_mae: 0.1251 - speed_output_mae: 0.1728 - val_loss: 0.0716 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0469 - val_angle_output_mae: 0.1268 - val_speed_output_mae: 0.1

173/173 [==============================] - 200s 1s/step - loss: 0.0625 - angle_output_loss: 0.0241 - speed_output_loss: 0.0383 - angle_output_mae: 0.1255 - speed_output_mae: 0.1507 - val_loss: 0.0682 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0434 - val_angle_output_mae: 0.1275 - val_speed_output_mae: 0.1222
Epoch 98/100
173/173 [==============================] - 284s 2s/step - loss: 0.0625 - angle_output_loss: 0.0241 - speed_output_loss: 0.0384 - angle_output_mae: 0.1253 - speed_output_mae: 0.1499 - val_loss: 0.0654 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0406 - val_angle_output_mae: 0.1276 - val_speed_output_mae: 0.1097
Epoch 99/100
173/173 [==============================] - 238s 1s/step - loss: 0.0620 - angle_output_loss: 0.0241 - speed_output_loss: 0.0379 - angle_output_mae: 0.1254 - speed_output_mae: 0.1491 - val_loss: 0.0785 - val_angle_output_loss: 0.0248 - val_speed_output_loss: 0.0537 - val_angle_output_mae: 0.1261 - val_speed_output_mae: 0.1

In [18]:
eval_results4 = model.evaluate(X_val, {'angle_output': y_angle_val, 'speed_output': y_speed_val})

87/87 [==============================] - 13s 150ms/step - loss: 0.0642 - angle_output_loss: 0.0248 - speed_output_loss: 0.0394 - angle_output_mae: 0.1264 - speed_output_mae: 0.0985
